In [1]:
clc
clear
close all
tic
%%
Pathname = ['D:\dataset\健康人数据' filesep 'data_64'] ;
Listing = dir(Pathname) ;
filename = {Listing(3:end).name}' ;
filt=1;
%ICA=1,ICA为高通
% 获取文件名并使用natsort进行排序
filename = natsort(filename);
EEG_data_37=[];
EEG_data_81=[];
EEG_data_525=[];
TFR=[];
for isSub =1:20
    isSub
    Pathname1 = [Pathname filesep filename{isSub}] ;
    Listing1 = dir(Pathname1) ;
    filename1 = {Listing1(3:end).name}' ;
    for days=1:3
        STD_list=[];
        STD_fre_list=[];
        STD_name_list=[];
        STD_In_list=[];
        STD_time_list=[];
        
        DEV_Fre_list=[];
        DEV_name_list=[];
        DEV_time_list=[];
        DEV_In_list=[];
        bad_epochs_idexs=[];
        remove_epochs=[];
        %% ASSR
        EEG = pop_loadset('filename','Filtered_data.set','filepath',[Pathname1 filesep filename1{days}] ) ;
        path_ASSR= [Pathname1 filesep filename1{days}] ;
        if EEG.nbchan==65
            EEG = pop_select( EEG, 'nochannel',{'EOG','BIP13'});
        else
        end
        EEG = eeg_checkset( EEG );
        EEG = pop_reref( EEG, [13 19]);
        EEG = eeg_checkset( EEG );
        EEG  = pop_basicfilter( EEG,  1:61 , 'Boundary', 'boundary', 'Cutoff',  50, 'Design', 'notch', 'Filter', 'PMnotch', 'Order',  180 ); % GUI: 17-Jan-2024 20:18:20
        EEG = eeg_checkset( EEG );
        EEG  = pop_basicfilter(EEG,  1:61 , 'Boundary', 'boundary', 'Cutoff',  30, 'Design', 'butter', 'Filter', 'highpass', 'Order', 8); % GUI: 17-Jan-2024 20:23:38
        EEG = eeg_checkset( EEG );
        EEG  = pop_basicfilter( EEG,  1:61 , 'Boundary', 'boundary', 'Cutoff',  44, 'Design', 'butter', 'Filter', 'lowpass', 'Order', 8 ); % GUI: 17-Jan-2024 20:20:29
        EEG = eeg_checkset( EEG );        
        EEG = pop_epoch( EEG, {  's11'  's12'  's13'  's14'  's15'  }, [-0.1  0.6], 'newname', ' resampled pruned with ICA epochs', 'epochinfo', 'yes');
        EEG = eeg_checkset( EEG );
        EEG = pop_rmbase( EEG, [-100 0] ,[]);
        EEG = eeg_checkset( EEG );
        % 设置阈值参数
        threshvalue = 100; % 阈值，单位：微伏
        a=1;
        for i = 1:size(EEG.data,3)
            segment = abs(squeeze(EEG.data(:,:,i))); % 获取当前段的数据
            for n=1:size(segment,2)
                % 如果超过阈值，则将当前段标记为坏段
                if segment(:,n) > threshvalue
                    bad_epochs_idexs(a)=i ;
                    %去除重合
                    bad_epochs_idexs=unique(bad_epochs_idexs);
                    bad_epochs_idexs = bad_epochs_idexs(bad_epochs_idexs ~= 0);
                    a=a+1;
                else
                end
            end
        end
        b=1;
        for bad_index=1:length(bad_epochs_idexs)-1
            bad_epochs=bad_epochs_idexs(bad_index);
            if strcmp(EEG.event( bad_epochs).type, 's11') && ~strcmp(EEG.event( bad_epochs+1).type, 's11')
                remove_epochs(b)=bad_epochs;
                remove_epochs(b+1)=bad_epochs+1;
                remove_epochs=unique(remove_epochs);
                remove_epochs= remove_epochs( remove_epochs~= 0);
                b=b+2;
            elseif ~strcmp(EEG.event( bad_epochs).type, 's11') && strcmp(EEG.event( bad_epochs-1).type, 's11')
                remove_epochs(b)=bad_epochs-1;
                remove_epochs(b+1)=bad_epochs;
                remove_epochs=unique(remove_epochs);
                remove_epochs= remove_epochs( remove_epochs~= 0);
                b=b+2;
            elseif strcmp(EEG.event( bad_epochs).type, 's11')&& strcmp(EEG.event( bad_epochs+1).type, 's11')
                remove_epochs(b)=bad_epochs;
                remove_epochs=unique(remove_epochs);
                remove_epochs= remove_epochs( remove_epochs~= 0);
                b=b+1;
            end
        end
        % 使用 unique() 函数去除重复元素
        unique_bad_epochs = unique(remove_epochs);
        EEG.data(:,:,unique_bad_epochs)=[];
        EEG.event(unique_bad_epochs)=[];
        EEG.epoch(unique_bad_epochs)=[];
        STD_num=1;
        STD_fre_num=1;
        STD_name_num=1;
        STD_In_num=1;
        STD_time_num=1;
        
        DEV_Fre_num=1;
        DEV_name_num=1;
        DEV_time_num=1;
        DEV_In_num=1;
        for list_epoch=1:size( EEG.data,3)-1
            if strcmp(EEG.event(list_epoch).type, 's11')  && strcmp(EEG.event( list_epoch+1).type, 's11')
                STD_list(STD_num)=list_epoch;
                STD_num=STD_num+1;
            elseif strcmp(EEG.event(list_epoch).type, 's12')
                DEV_Fre_list(DEV_Fre_num)=list_epoch;
                DEV_Fre_num=DEV_Fre_num+1;
                STD_fre_list(STD_fre_num)=list_epoch-1;
                STD_fre_num=STD_fre_num+1;
            elseif strcmp(EEG.event(list_epoch).type, 's13')
                DEV_name_list(DEV_name_num)=list_epoch;
                DEV_name_num=DEV_name_num+1;
                STD_name_list(STD_name_num)=list_epoch-1;
                STD_name_num=STD_name_num+1;
            elseif strcmp(EEG.event(list_epoch).type, 's14')
                DEV_time_list(DEV_time_num)=list_epoch;
                DEV_time_num=DEV_time_num+1;
                STD_time_list(STD_time_num)=list_epoch-1;
                STD_time_num=STD_time_num+1;
            elseif strcmp(EEG.event(list_epoch).type, 's15')
                DEV_In_list(DEV_In_num)=list_epoch;
                DEV_In_num=DEV_In_num+1;
                STD_In_list(STD_In_num)=list_epoch-1;
                STD_In_num=STD_In_num+1;
            end
        end
        STD_37=EEG.data(:,:,STD_list);
        channel=EEG.chanlocs;
        %% 81
        STD_list=[];
        STD_fre_list=[];
        STD_name_list=[];
        STD_In_list=[];
        STD_time_list=[];
        
        DEV_Fre_list=[];
        DEV_name_list=[];
        DEV_time_list=[];
        DEV_In_list=[];
        bad_epochs_idexs=[];
        remove_epochs=[];
        EEG = pop_loadset('filename','Filtered_data.set','filepath',[Pathname1 filesep filename1{days}] ) ;
        if EEG.nbchan==65
            EEG = pop_select( EEG, 'nochannel',{'EOG','BIP13'});
        else
        end
        EEG = eeg_checkset( EEG );
        EEG = pop_reref( EEG, [13 19]);
        EEG = eeg_checkset( EEG );
        EEG  = pop_basicfilter( EEG,  1:61 , 'Boundary', 'boundary', 'Cutoff',  50, 'Design', 'notch', 'Filter', 'PMnotch', 'Order',  180 ); % GUI: 17-Jan-2024 20:18:20
        EEG = eeg_checkset( EEG );
        EEG  = pop_basicfilter(EEG,  1:61 , 'Boundary', 'boundary', 'Cutoff',  75, 'Design', 'butter', 'Filter', 'highpass', 'Order',  8, 'RemoveDC', 'on' ); % GUI: 17-Jan-2024 20:23:38
        EEG = eeg_checkset( EEG );
        EEG  = pop_basicfilter( EEG,  1:61 , 'Boundary', 'boundary', 'Cutoff',  90, 'Design', 'butter', 'Filter', 'lowpass', 'Order', 8, 'RemoveDC', 'on' ); % GUI: 17-Jan-2024 20:20:29
        EEG = eeg_checkset( EEG );       
        %%
        EEG = pop_epoch( EEG, {  's11'  's12'  's13'  's14'  's15'  }, [-0.1  0.6], 'newname', ' resampled pruned with ICA epochs', 'epochinfo', 'yes');
        EEG = eeg_checkset( EEG );
        EEG = pop_rmbase( EEG, [-100 0] ,[]);
        EEG = eeg_checkset( EEG );
        % 设置阈值参数
        threshvalue = 100; % 阈值，单位：微伏
        a=1;
        for i = 1:size(EEG.data,3)
            segment = abs(squeeze(EEG.data(:,:,i))); % 获取当前段的数据
            for n=1:size(segment,2)
                % 如果超过阈值，则将当前段标记为坏段
                if segment(:,n) > threshvalue
                    bad_epochs_idexs(a)=i ;
                    %去除重合
                    bad_epochs_idexs=unique(bad_epochs_idexs);
                    bad_epochs_idexs = bad_epochs_idexs(bad_epochs_idexs ~= 0);
                    a=a+1;
                else
                end
            end
        end
        b=1;
        for bad_index=1:length(bad_epochs_idexs)-1
            bad_epochs=bad_epochs_idexs(bad_index);
            if strcmp(EEG.event( bad_epochs).type, 's11') && ~strcmp(EEG.event( bad_epochs+1).type, 's11')
                remove_epochs(b)=bad_epochs;
                remove_epochs(b+1)=bad_epochs+1;
                remove_epochs=unique(remove_epochs);
                remove_epochs= remove_epochs( remove_epochs~= 0);
                b=b+2;
            elseif ~strcmp(EEG.event( bad_epochs).type, 's11') && strcmp(EEG.event( bad_epochs-1).type, 's11')
                remove_epochs(b)=bad_epochs-1;
                remove_epochs(b+1)=bad_epochs;
                remove_epochs=unique(remove_epochs);
                remove_epochs= remove_epochs( remove_epochs~= 0);
                b=b+2;
            elseif strcmp(EEG.event( bad_epochs).type, 's11')&& strcmp(EEG.event( bad_epochs+1).type, 's11')
                remove_epochs(b)=bad_epochs;
                remove_epochs=unique(remove_epochs);
                remove_epochs= remove_epochs( remove_epochs~= 0);
                b=b+1;
            end
        end
        % 使用 unique() 函数去除重复元素
        unique_bad_epochs = unique(remove_epochs);
        EEG.data(:,:,unique_bad_epochs)=[];
        EEG.event(unique_bad_epochs)=[];
        EEG.epoch(unique_bad_epochs)=[];
        STD_num=1;
        STD_fre_num=1;
        STD_name_num=1;
        STD_In_num=1;
        STD_time_num=1;
        
        DEV_Fre_num=1;
        DEV_name_num=1;
        DEV_time_num=1;
        DEV_In_num=1;
        for list_epoch=1:size( EEG.data,3)-1
            if strcmp(EEG.event(list_epoch).type, 's11')  && strcmp(EEG.event( list_epoch+1).type, 's11')
                STD_list(STD_num)=list_epoch;
                STD_num=STD_num+1;
            elseif strcmp(EEG.event(list_epoch).type, 's12')
                DEV_Fre_list(DEV_Fre_num)=list_epoch;
                DEV_Fre_num=DEV_Fre_num+1;
                STD_fre_list(STD_fre_num)=list_epoch-1;
                STD_fre_num=STD_fre_num+1;
            elseif strcmp(EEG.event(list_epoch).type, 's13')
                DEV_name_list(DEV_name_num)=list_epoch;
                DEV_name_num=DEV_name_num+1;
                STD_name_list(STD_name_num)=list_epoch-1;
                STD_name_num=STD_name_num+1;
            elseif strcmp(EEG.event(list_epoch).type, 's14')
                DEV_time_list(DEV_time_num)=list_epoch;
                DEV_time_num=DEV_time_num+1;
                STD_time_list(STD_time_num)=list_epoch-1;
                STD_time_num=STD_time_num+1;
            elseif strcmp(EEG.event(list_epoch).type, 's15')
                DEV_In_list(DEV_In_num)=list_epoch;
                DEV_In_num=DEV_In_num+1;
                STD_In_list(STD_In_num)=list_epoch-1;
                STD_In_num=STD_In_num+1;
            end
        end
        STD_81=EEG.data(:,:,STD_list);
        channel=EEG.chanlocs;
%        fileDir=[Pathname1 filesep filename1{days}];
 %       savepath=strcat(fileDir,'\ASSR_epoched');%拼接路径和文件名
  %      save(savepath,'STD_37','STD_81','channel')%指定存储路径 存储名称和存储进去的变量
    end
end